# Run and plot COPSE Reloaded output

In [ ]:
#Configure width of Jupyter notebook in browser
#https://discourse.julialang.org/t/decrease-margins-of-ijulia-in-jupyter-notebook/6912
# display("text/html", "<style>.container { width:100% !important; }</style>")
#Increase width used by Julia output eg DataFrame
ENV["COLUMNS"]=160
ENV["LINES"] = 1000  # show up to 1000 lines of datatable output

# activate the correct environment
import Pkg
# NB: jupyter still seems to separately precompile from the REPL ?
Pkg.activate("../")

using Revise
using Plots; plotlyjs(size=(750, 565));

In [ ]:
using DataFrames


# Import PALEO modules

import PALEOboxes as PB

import PALEOmodel

import PALEOcopse


## Call copse_reloaded_expts to create run object parameterized for COPSE reloaded baseline case

In [ ]:
include("copse_reloaded_reloaded_expts.jl")

model = copse_reloaded_reloaded_expts("reloaded", ["baseline"])


## Call PALEOmodel.initialize! to get initial_state vector, and modeldata storage struct

In [ ]:
initial_state, modeldata = PALEOmodel.initialize!(model)

## Optional step useful for debugging: check model derivative

In [ ]:
# call ODE function to check derivative
initial_deriv = similar(initial_state)

PALEOmodel.SolverFunctions.ModelODE(modeldata)(
    initial_deriv, 
    initial_state, 
    nothing,
    0.0
)

println("initial_state", initial_state)
println("initial_deriv", initial_deriv)

println()
for (var, var_sms) in zip(PB.get_vars(modeldata.solver_view_all.stateexplicit), PB.get_vars(modeldata.solver_view_all.stateexplicit_deriv))
    var_data = PB.get_data(var, modeldata)
    var_sms_data = PB.get_data(var_sms, modeldata)
    println(
        var.name, "\t", var_data[], "\t", var_sms.name, "\t", var_sms_data[],
        "\ttimescale (total) (yr)\t", abs(PB.get_total(var_data[])/PB.get_total(var_sms_data[]))
    )
end

## Integrate model as an ODE

In [ ]:
println("integrate, ODE")

paleorun = PALEOmodel.Run(model=model, output = PALEOmodel.OutputWriters.OutputMemory())

# first run includes JIT time
@time PALEOmodel.ODE.integrate(
    paleorun, initial_state, modeldata, (-1000e6, 0), 
    solvekwargs=(
        reltol=1e-4,
    )
); 
  

## Plot output

In [ ]:
copse_reloaded_reloaded_plot(paleorun.output);

## Compare against archived model output from Lenton, Daines, Mills (2018)

In [ ]:
# load archived model output
include("compare_output.jl")

comparedata = CompareOutput.copse_output_load("reloaded","reloaded_baseline")

diff = CompareOutput.compare_copse_output(paleorun.output, comparedata)
firstpoint=50
sort(DataFrames.describe(diff[firstpoint:end, :], :std, :min, :max, :mean), :std)

## Display model parameters
This illustrates the modularised model structure, with:
- Domains global, atm, land, ocean, oceansurface, oceanfloor, sedcrust containing forcings and biogeochemical Reactions, with Parameters attached to each Reaction.
- Additional Domains fluxAtoLand, fluxLandtoSedCrust, fluxOceanBurial, fluxOceanfloor, fluxRtoOcean, fluxSedCrusttoAOcean containing flux coupler Reactions.

In [ ]:

PB.show_parameters(model)

## Display model Variables
This illustrates the modularized model structure, with:
- Domains global, atm, land, ocean, oceansurface, oceanfloor, sedcrust containing Variables linked to Reactions (either property-dependencies or target-contributors pairs).
- Additional Domains fluxAtoLand, fluxLandtoSedCrust, fluxOceanBurial, fluxOceanfloor, fluxRtoOcean, fluxSedCrusttoAOcean containing target-contributor pairs representing inter-module fluxes.

In [ ]:
ENV["COLUMNS"]=600
PB.show_variables(model, modeldata=modeldata, showlinks=true)

## Display model output

In [ ]:
# output is a Dict of OutputMemoryDomains, data field is a DataFrame with output for that Domain
println("paleorun.domains keys: ", keys(paleorun.output.domains))

# list fields in 'atm' Domain
println("output[\"atm\"] names: ", names(paleorun.output.domains["atm"].data))

# show a subset of output fields
paleorun.output.domains["atm"].data[!, [:tmodel, :pCO2atm, :pCO2PAL]]

## Change climate sensitivity from default 3C to 6C and compare pCO2 prediction (see Lenton etal (2018) Figure 11)

In [ ]:
# rerun and keep baseline output for comparison

PB.setvalue!(PB.get_reaction(model, "global", "temp_global").pars.k_c, 4.328)
paleorun = PALEOmodel.Run(model=model, output = PALEOmodel.OutputWriters.OutputMemory())
PALEOmodel.ODE.integrate(
    paleorun, initial_state, modeldata, (-1000e6, 0),
    solvekwargs=(
        reltol=1e-4,
    )
)
output_baseline_3C = paleorun.output

PB.setvalue!(PB.get_reaction(model, "global", "temp_global").pars.k_c, 8.656)
paleorun = PALEOmodel.Run(model=model, output = PALEOmodel.OutputWriters.OutputMemory())
PALEOmodel.ODE.integrate(
    paleorun, initial_state, modeldata, (-1000e6, 0),
    solvekwargs=(
        reltol=1e-4,
    )
)
output_baseline_6C = paleorun.output


In [ ]:
# model-model plot 
copse_reloaded_reloaded_plot([output_baseline_6C, output_baseline_3C]);


## Overlay comparison data (data compilations available on request, not publically distributable)

In [ ]:
# compare to data
# include("compare_output_plots.jl")
# copse_reloaded_comparedata([output_baseline_6C, output_baseline_3C], include_Sr=true)


# Additional diagnostic checks

In [ ]:
PB.show_methods_setup(model)

In [ ]:
PB.show_methods_initialize(model)

In [ ]:
PB.show_methods_do(model)

In [ ]:
# Get state variables

println()
println("state and sms vars:")
println()

for (var, var_sms) in zip(PB.get_vars(modeldata.solver_view_all.stateexplicit), PB.get_vars(modeldata.solver_view_all.stateexplicit_deriv)) 
    println(var.name, "\t\t", PB.get_var_type(var), "\t\t", PB.get_attribute(var, :units), "\t\t", PB.get_attribute(var, :description))
    println(var_sms.name, "\t\t", PB.get_var_type(var_sms), "\t\t", PB.get_attribute(var_sms, :units), "\t\t", PB.get_attribute(var_sms, :description))
end


const expected_num_state_variables = 11
num_state_variables = PB.num_vars(modeldata.solver_view_all.stateexplicit)

println("num_state_variables = ", num_state_variables)
@PB.TestUtils.check_true num_state_variables == expected_num_state_variables


In [ ]:
# Check variable access and metadata

# Create a DataFrame of filtered VariableDomain objects
# define a function to filter variables
filterstatevars(attrb) = attrb[:vfunction] == PB.VF_StateExplicit
PB.show_variables(model, modeldata=modeldata, filter=filterstatevars, showlinks=true)[:, [:name, :units, :vfunction, :description, :data, :dependencies]]
